In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect

In [7]:
factory = PokemonFactory("pokemon.json")
df = pd.read_json("pokemon.json")
pokemons = list(df.columns)
pokeballs = ["pokeball", "ultraball", "fastball", "heavyball"]

In [84]:
def estimate_catchrate(pokemon_instance, pokeball, noise, n):
    return np.average([attempt_catch(pokemon_instance, pokeball, noise)[0] for _ in range(n)])

### a) Ejecutando la función 100 veces, para cada Pokemon en condiciones ideales (HP:100 %, LVL 100) ¿Cuál es la probabilidad de captura promedio para cada pokebola?

In [85]:
data = {}
for pokemon in pokemons:
    bicho = factory.create(pokemon, 100, StatusEffect.NONE, 1)
    data[pokemon] = {}
    for ball in pokeballs:
        data[pokemon][ball] = estimate_catchrate(bicho, ball, 0, 3000)
print(data)

{'jolteon': {'pokeball': 0.058, 'ultraball': 0.12166666666666667, 'fastball': 0.24533333333333332, 'heavyball': 0.030666666666666665}, 'caterpie': {'pokeball': 0.332, 'ultraball': 0.666, 'fastball': 0.33466666666666667, 'heavyball': 0.30366666666666664}, 'snorlax': {'pokeball': 0.029, 'ultraball': 0.059666666666666666, 'fastball': 0.030333333333333334, 'heavyball': 0.072}, 'onix': {'pokeball': 0.060333333333333336, 'ultraball': 0.119, 'fastball': 0.066, 'heavyball': 0.08233333333333333}, 'mewtwo': {'pokeball': 0.0036666666666666666, 'ultraball': 0.007666666666666666, 'fastball': 0.014333333333333333, 'heavyball': 0.002}}


In [86]:
average_rates = {
    ball: np.average([values[ball] for values in data.values()]) for ball in pokeballs
}
errors = {
    ball: np.std([values[ball] for values in data.values()]) for ball in pokeballs
}

df = pd.DataFrame({'Pokeball': pokeballs,
                   'Average Rate': list(average_rates.values()),
                   'Error': list(errors.values())})

print(df)

    Pokeball  Average Rate     Error
0   pokeball      0.096600  0.119520
1  ultraball      0.194800  0.239334
2   fastball      0.138133  0.128271
3  heavyball      0.098133  0.106743


In [87]:
fig = px.bar(
    data_frame=df,
    x="Pokeball", 
    y="Average Rate", 
    title="Average capture probability by Pokeball",
    labels={
        "x": "Pokeball type",
        "y": "Average capture rate"
    },
    error_y="Error"
)
fig.show()

### 1b) ¿Es cierto que algunas pokebolas son más o menos efectivas dependiendo de propiedades intrinsecas de cada Pokemon? Justificar.

In [88]:
rates_by_pokemon = {
    pokemon: {
        pokeball: data[pokemon][pokeball] / data[pokemon]["pokeball"] for pokeball in data[pokemon].keys() if pokeball != "pokeball"
    } for pokemon in data.keys()
}
print(rates_by_pokemon)

{'jolteon': {'ultraball': 2.0977011494252875, 'fastball': 4.229885057471264, 'heavyball': 0.528735632183908}, 'caterpie': {'ultraball': 2.0060240963855422, 'fastball': 1.008032128514056, 'heavyball': 0.9146586345381524}, 'snorlax': {'ultraball': 2.057471264367816, 'fastball': 1.0459770114942528, 'heavyball': 2.482758620689655}, 'onix': {'ultraball': 1.9723756906077345, 'fastball': 1.0939226519337018, 'heavyball': 1.3646408839779005}, 'mewtwo': {'ultraball': 2.090909090909091, 'fastball': 3.909090909090909, 'heavyball': 0.5454545454545455}}


In [89]:
table = []
for pokemon in rates_by_pokemon.keys():
    for pokeball in rates_by_pokemon[pokemon].keys():
        table.append([pokemon, pokeball, rates_by_pokemon[pokemon][pokeball]])
df = pd.DataFrame(table, columns=["pokemon", "pokeball", "rate"])
print(df)

     pokemon   pokeball      rate
0    jolteon  ultraball  2.097701
1    jolteon   fastball  4.229885
2    jolteon  heavyball  0.528736
3   caterpie  ultraball  2.006024
4   caterpie   fastball  1.008032
5   caterpie  heavyball  0.914659
6    snorlax  ultraball  2.057471
7    snorlax   fastball  1.045977
8    snorlax  heavyball  2.482759
9       onix  ultraball  1.972376
10      onix   fastball  1.093923
11      onix  heavyball  1.364641
12    mewtwo  ultraball  2.090909
13    mewtwo   fastball  3.909091
14    mewtwo  heavyball  0.545455


In [106]:
fig = px.bar(
    data_frame=df,
    x="pokemon", 
    y="rate", 
    title="Effectiveness of Special Pokeballs by Pokemon",
    labels={
        "pokemon": "Pokemon",
        "rate": "Effectiveness relative to a regular pokeball"
    },
    barmode="group",
    color="pokeball"
)
fig.show()

Podemos ver desde el grafico que es cierto que algunos tipos de pokebolas son (en comparacion a la pokebola base) más efectivas en ciertos pokemones. Por ejemplo, las fastball son mucho mejores contra pokemons como Jolteon y Mewtwo, y las heavyballs son mejores contra Snorlax, pero peores contra Jolteon o Mewtwo.
En cambio, la ultraball no parece depender del pokemon a atrapar.

### 2a)¿Las condiciones de salud tienen algún efecto sobre la efectividad de la captura? Si es ası́, ¿Cuál es más o menos efectiva?

In [168]:
pokemon_test = "jolteon"
variants = { 
    status: factory.create(pokemon_test, 100, status, 1) for status in StatusEffect 
}
catchrate_by_variant = { 
    status: estimate_catchrate(pokemon, pokeballs[0], 1, 3000) for (status, pokemon) in variants.items() 
}
print(catchrate_by_variant)

{<StatusEffect.POISON: ('poison', 1.5)>: 0.09766666666666667, <StatusEffect.BURN: ('burn', 1.5)>: 0.095, <StatusEffect.PARALYSIS: ('paralysis', 1.5)>: 0.084, <StatusEffect.SLEEP: ('sleep', 2)>: 0.12166666666666667, <StatusEffect.FREEZE: ('freeze', 2)>: 0.13066666666666665, <StatusEffect.NONE: ('none', 1)>: 0.06233333333333333}


In [101]:
fig = px.bar(
    x = list(map(lambda status : status.name, catchrate_by_variant.keys())), 
    y = list(catchrate_by_variant.values()), 
    title = "Average catch rate by Status Effect",
    labels= {
        "x": "Status Effect",
        "y": "Catch Rate"
    }
)
fig.show()

Podemos ver que los más efectivos son SLEEP y FREEZE. Mientras que los menos efectivos son los otros tres.

### 2b) ¿Cómo afectan los puntos de vida a la efectividad de la captura? Sugerencia: Elegir uno o dos Pokemones y manteniendo el resto de los parámetros constantes, calcular la probabilidad de captura para distintos HP %

In [126]:
def get_catchrate_with_error(pokemon_instance, pokeball, noise, n):
    values = [attempt_catch(pokemon_instance, pokeball, noise)[1] for _ in range(n)]
    return {
        "rate": np.average(values), 
        "error": np.std(values)
    }

In [130]:
import decimal

def drange(x, y, jump):
  while x < y + jump:
    yield float(x)
    x += decimal.Decimal(jump)

health_vars = { 
    percentage: factory.create(pokemon_test, 100, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) 
}

catchrate_by_health = { 
    percentage: get_catchrate_with_error(pokemon, pokeballs[0], 0.15, 3000) for (percentage, pokemon) in health_vars.items() 
}
print(catchrate_by_health)

{0.0: {'rate': 0.174917070551499, 'error': 0.02651908296415759}, 0.05: {'rate': 0.17073237853297416, 'error': 0.025881785007621862}, 0.1: {'rate': 0.1651288510620135, 'error': 0.024172693368430666}, 0.15000000000000002: {'rate': 0.15848333599329187, 'error': 0.024237505091845166}, 0.2: {'rate': 0.15248131294271505, 'error': 0.02316917722849605}, 0.25: {'rate': 0.14717445893253062, 'error': 0.022153510287497388}, 0.30000000000000004: {'rate': 0.14089609493041236, 'error': 0.02122774994816174}, 0.35000000000000003: {'rate': 0.13548677355386973, 'error': 0.020438907750933627}, 0.4: {'rate': 0.12909740909068038, 'error': 0.01914688912453892}, 0.45: {'rate': 0.12435205749910709, 'error': 0.018245992467525306}, 0.5: {'rate': 0.11821385157335973, 'error': 0.017671076174871733}, 0.55: {'rate': 0.11115419521459448, 'error': 0.017043458574797173}, 0.6000000000000001: {'rate': 0.10579426030541167, 'error': 0.01629649573480681}, 0.65: {'rate': 0.09982501388493409, 'error': 0.015301473449061631}, 0

In [138]:
values = {
    "x": [],
    "y": [],
    "upper_y": [],
    "lower_y": []
}
for item in catchrate_by_health.items():
    values["x"].append(item[0])
    values["y"].append(item[1]["rate"])
    values["upper_y"].append(item[1]["rate"] + item[1]["error"])
    values["lower_y"].append(item[1]["rate"] - item[1]["error"])
print(values)

{'x': [0.0, 0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0], 'y': [0.174917070551499, 0.17073237853297416, 0.1651288510620135, 0.15848333599329187, 0.15248131294271505, 0.14717445893253062, 0.14089609493041236, 0.13548677355386973, 0.12909740909068038, 0.12435205749910709, 0.11821385157335973, 0.11115419521459448, 0.10579426030541167, 0.09982501388493409, 0.09465581925671586, 0.08817160580431418, 0.08277888085095778, 0.0767277682684424, 0.0712825955503526, 0.06462865814422367, 0.058892520469756654], 'upper_y': [0.20143615351565658, 0.19661416354059602, 0.18930154443044417, 0.18272084108513703, 0.1756504901712111, 0.16932796922002802, 0.1621238448785741, 0.15592568130480336, 0.14824429821521928, 0.1425980499666324, 0.13588492774823147, 0.12819765378939166, 0.12209075604021848, 0.11512648733399572, 0.10861249755717432, 0.101

In [152]:
import plotly.graph_objs as go

fig = go.Figure([
    go.Scatter(
        x=values["x"],
        y=values["y"],
        line=dict(color='rgb(0,100,80)'),
        mode='lines',
        showlegend=False,
        name="average rate"
    ),
    go.Scatter(
        x=values["x"]+values["x"][::-1], # x, then x reversed
        y=values["upper_y"]+values["lower_y"][::-1], # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False,
    )
])
fig.update_layout(
    title = f"Capture rate of {pokemon_test} vs its Health Points",
    xaxis_title="HP",
    yaxis_title="Catch rate",
)
fig.update_yaxes(
    rangemode="tozero",
    tickformat=".1%",
)
fig.update_xaxes(
    tickformat=".0%"
)
fig.show()

Podemos ver que a medida que aumentamos la vida del pokemon, menor será la posibilidad de captura

### 2c) ¿Qué parámetros son los que más afectan la probabilidad de captura?

Para ver que parámetros afectan más a la probabilidad de captura, calculamos la varianza de los valores calculados variando el parámetro y manteniendo los otros constantes.
Los parametros son:
- La vida
- El Efecto de estado
- La pokebola
Tomamos un Jolteon como ejemplo.

In [169]:
data_status = []
data_hp = []
data_pb = []
for hp in range(10):
    data_hp.append(attempt_catch(
        factory.create(pokemon_test, 100, StatusEffect.NONE, hp*0.1),
        pokeballs[0]
    )[1])
for status in StatusEffect:
    data_status.append(attempt_catch(
        factory.create(pokemon_test, 100, status, 1),
        pokeballs[0]
    )[1])
for pb in pokeballs:
    data_pb.append(attempt_catch(
        factory.create(pokemon_test, 100, StatusEffect.NONE, 1),
        pb
    )[1])

data_var = [np.var(data_status), np.var(data_hp), np.var(data_pb)]
df = pd.DataFrame(data_var,index=['Status Effect','HP','Pokeball'],columns=['Variance'])
px.bar(df,x=df.index,y="Variance",title="Effect of changing parameters on catch effectiveness",labels={
    "index": "Parameter"
}).show()

Para Jolteon en particular, la mayor varianza se obtiene cambiando el tipo de pokebola que se usa. Debido a que la efectividad de una pokebola depende de los atributos del pokemon, esto puede cambiar.

In [181]:
# Funcion devuelve una tupla (varianza_por_tipo, max y minimo por tipo)
def calculate_all_variances(pokemon, level, iterations):
    health_vars = { percentage: factory.create(pokemon_test, level, StatusEffect.NONE, percentage) for percentage in drange(0, 1, 0.05) }
    catchrate_by_health = { percentage: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (percentage, pokemon) in health_vars.items() }

    variants = { status: factory.create(pokemon, level, status, 1) for status in StatusEffect }
    catchrate_by_variant = { status.name: estimate_catchrate(pokemon, pokeballs[0], 1, iterations) for (status, pokemon) in variants.items() }

    healthy_pokemon = factory.create(pokemon, level, StatusEffect.NONE, 1)
    catchrate_by_pokeball = { pokeball: estimate_catchrate(healthy_pokemon, pokeball, 1, 3000) for pokeball in pokeballs }
    return {
        "health": np.var(list(catchrate_by_health.values())),
        "status": np.var(list(catchrate_by_variant.values())),
        "pokeball": np.var(list(catchrate_by_pokeball.values()))
    }

all_pokemons_variances = { pokemon: calculate_all_variances(pokemon, 100, 3000) for pokemon in pokemons }

df = pd.DataFrame.from_dict(all_pokemons_variances)
df = df.assign(average=df.mean(axis=1))
print(df)
fig5 = px.bar(df, barmode="group",title="Effect of changing parameters on catch effectiveness by pokemon",labels={
    "index": "Parameter",
    "value": "Variance",
    "variable": "Pokemon"
})
fig5.show()

           jolteon  caterpie   snorlax      onix    mewtwo   average
health    0.001405  0.001472  0.001590  0.001474  0.001328  0.001454
status    0.000519  0.005637  0.000143  0.000650  0.000003  0.001390
pokeball  0.005871  0.012058  0.000473  0.000748  0.000045  0.003839


El gráfico nos muestra cuáles son las varianzas por cada pokemon. Podemos ver que caterpie es afectado mucho más por el estado o la pokebola, mientras que mewtwo no es prácticamente afectado por estos parámetros. Por otro lado, la incidencia de la vida del pokemon en la efectividad no parece depender del pokemon en cuestión.
Agarrando el promedio de todas las varianzas, parece que la vida y la pokebola afectan bastante en comparacion al estado y nivel.

### 2d) Teniendo en cuenta uno o dos pokemones distintos: ¿Qué combinación de condiciones (propiedades mutables) y pokebola conviene utilizar para capturarlos?

In [200]:
max_catchrate = {
    pokemon: { "rate" : 0 } for pokemon in pokemons
}
for status in StatusEffect:
    for pokeball in pokeballs:
        for pokemon in max_catchrate.keys():
            catchrate = attempt_catch(
                factory.create(pokemon, 100, status, 1),
                pokeball
            )[1]
            if max_catchrate[pokemon]["rate"] < catchrate:
                max_catchrate[pokemon] = {
                    "rate": catchrate,
                    "status": status.name,
                    "pokeball": pokeball,
                    "HP": 1
                }
print(max_catchrate)
            

{'jolteon': {'rate': 0.4688, 'status': 'SLEEP', 'pokeball': 'fastball', 'HP': 1}, 'caterpie': {'rate': 1, 'status': 'SLEEP', 'pokeball': 'ultraball', 'HP': 1}, 'snorlax': {'rate': 0.1693, 'status': 'SLEEP', 'pokeball': 'heavyball', 'HP': 1}, 'onix': {'rate': 0.2344, 'status': 'SLEEP', 'pokeball': 'ultraball', 'HP': 1}, 'mewtwo': {'rate': 0.0313, 'status': 'SLEEP', 'pokeball': 'fastball', 'HP': 1}}


<insert blabla>

### 2e) A partir del punto anterior, ¿serı́a efectiva otra combinación de parámetros teniendo en cuenta un nivel del pokemon más bajo (o más alto)?

Repetimos el anterior pero con un nivel de 40 en vez de 100

In [204]:
max_catchrate = {
    pokemon: { "rate" : 0 } for pokemon in pokemons
}
for status in StatusEffect:
    for pokeball in pokeballs:
        for pokemon in max_catchrate.keys():
            catchrate = attempt_catch(
                factory.create(pokemon, 40, status, 1),
                pokeball
            )[1]
            if max_catchrate[pokemon]["rate"] < catchrate:
                max_catchrate[pokemon] = {
                    "rate": catchrate,
                    "status": status.name,
                    "pokeball": pokeball,
                    "HP": 1
                }
print(max_catchrate)

{'jolteon': {'rate': 0.4688, 'status': 'SLEEP', 'pokeball': 'fastball', 'HP': 1}, 'caterpie': {'rate': 1, 'status': 'SLEEP', 'pokeball': 'ultraball', 'HP': 1}, 'snorlax': {'rate': 0.1693, 'status': 'SLEEP', 'pokeball': 'heavyball', 'HP': 1}, 'onix': {'rate': 0.2344, 'status': 'SLEEP', 'pokeball': 'ultraball', 'HP': 1}, 'mewtwo': {'rate': 0.0313, 'status': 'SLEEP', 'pokeball': 'fastball', 'HP': 1}}


Parece que el nivel no afecta qué parametros usar.